In [1]:
word_to_number = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5
}

import numpy as np
import random
import torch
torch.set_float32_matmul_precision('medium')

alpha = 0.10

import json
import math
import pandas as pd
from matplotlib import pyplot as plt
import os, sys

# !git clone https://github.com/zlin7/LVD.git
%cd ../LVD
# !pip install ipdb

c:\Users\huanx\python\LLMCP\LVD


C:\Users\huanx\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import time
import tracemalloc

In [3]:
import json
import pandas as pd
import math
from data.preprocess_small_datasets import pretrain_general
from models.regmodel import KernelMLKR
from models.conformal import LocalConditional, PIConstructor
import numpy as np
import os
import random
import psutil
import time


def range_modification(y_qlow, y_qup, range_low,  range_up):
    y_qlow = np.clip(y_qlow, range_low, range_up)
    y_qup = np.clip(y_qup, range_low, range_up)
    return y_qlow, y_qup

def run_experiment(X, y, seed, dataset, dimension):
    random.seed(seed)
    np.random.seed(seed)

    # X = clr_transform(X)
    X = X.to_numpy().astype(np.float32)
    y = y.to_numpy().astype(np.float32)

    from sklearn.model_selection import train_test_split
    X_cal, X_test, y_cal, y_test = train_test_split(X, y, test_size=0.5, random_state=seed)

    # X_cal = X_cal[:int(len(X_cal) * cal_size)]
    # y_cal = y_cal[:int(len(y_cal) * cal_size)]
    
    y_cal = y_cal.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    DNN_model, readout_layer = pretrain_general(X_cal, y_cal, seed=0, quiet=True, model_setting=0)
    embed_cal = DNN_model.embed(X_cal)
    embed_test = DNN_model.embed(X_test)

    kernel_model = KernelMLKR(d=10, seed=0, n_iters=500, norm=True, lr=1e-3)
    kernel_model.fit(embed_cal, y_cal.flatten()) 

    lvd = LocalConditional(K_obj=kernel_model)
    lvd.fit(embed_cal, y_cal.flatten(), m=readout_layer)

    results = lvd.eval(embed_test, y_test.flatten(), lvd.PI, alpha=alpha, quiet=False)

    y_qlow = results['lo'].to_numpy()
    y_qup = results['hi'].to_numpy()
    y_qlow, y_qup = range_modification(y_qlow, y_qup, 1, 5)

    intervals = [[(low, high)] for low, high in zip(y_qlow, y_qup)]
    print(intervals)
    
    df = pd.DataFrame({
        'low':    [iv[0][0] for iv in intervals],
        'up':     [iv[0][1] for iv in intervals],
        'y_test': y_test.flatten()
    })
    
    df.to_csv(f'LVD_{dataset}_{dimension}_{seed}.csv', index=False)

    adjusted_intervals = [
    [
        (
            next((num for num in [1, 2, 3, 4, 5] if abs(low - num) < 0.1), low),
            next((num for num in [1, 2, 3, 4, 5] if abs(high - num) < 0.1), high)
        )
        for low, high in sample_intervals
    ]
    for sample_intervals in intervals]

    intervals = adjusted_intervals

    in_interval = [
        any(low <= true_value <= high for low, high in sample_intervals)
        for sample_intervals, true_value in zip(intervals, y_test)
    ]

    coverage_rate = np.mean(in_interval)
    average_width = np.mean([high - low for sample_intervals in intervals for low, high in sample_intervals])  

    print(f"Seed: {seed}, Width: {average_width:.4f}, Coverage: {coverage_rate:.4f}")

    return average_width, coverage_rate

def calculate_statistics(X, y, num_runs=100, seed_start=1, dataset='Summeval', dimension='consistency'):
    from tqdm import tqdm
    timecost = []
    memory = []
    for i in tqdm(range(num_runs), desc="Running experiments"):
        seed = seed_start + i
        tracemalloc.start()
        start = time.perf_counter()
        average_width, coverage_rate = run_experiment(X, y, seed, dataset, dimension)
        end = time.perf_counter()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        timecost.append(end - start)
        memory.append(peak)

    mean_time = np.mean(timecost)
    std_time = np.std(timecost)
    mean_memory = np.mean(memory)
    std_memory = np.std(memory)

    return  mean_time, std_time, mean_memory, std_memory

In [4]:
import os
import pandas as pd

folder_path = f'../model_logits/qwen/'
for dimension in ["cosmos", "drop", "esnli", "gsm8k"]:
        file_path = os.path.join(folder_path, f"SocREval_{dimension}_logits.csv")
        df = pd.read_csv(file_path)
        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]
        mean_time, std_time, mean_memory, std_memory =  calculate_statistics(X, y, num_runs=10, seed_start=1, dimension=dimension, dataset='SocREval')



Running experiments:   0%|          | 0/10 [00:00<?, ?it/s]

Setting seeds to 0


fit: 100%|██████████| 97/97 [00:00<00:00, 6928.60it/s]


MSE=1.3509423762804524


Running experiments:  10%|█         | 1/10 [01:18<11:49, 78.87s/it]

...

[[(1.0, 5.0)], [(1.9999990463256836, 5.0)], [(2.0, 5.0)], [(2.7388486862182617, 5.0)], [(2.0, 5.0)], [(1.0, 4.4412713050842285)], [(1.0, 3.345366597175598)], [(1.9999990463256836, 5.0)], [(2.0, 5.0)], [(2.0, 5.0)], [(2.0, 5.0)], [(2.0, 5.0)], [(2.0, 5.0)], [(1.9999990463256836, 5.0)], [(1.9999990463256836, 5.0)], [(2.0, 5.0)], [(1.0, 4.9999988079071045)], [(1.0, 4.6382105350494385)], [(1.0, 4.425367951393127)], [(1.0, 4.2002105712890625)], [(1.9999980926513672, 5.0)], [(1.9999990463256836, 5.0)], [(2.0, 5.0)], [(1.9999995231628418, 5.0)], [(1.0, 4.633852481842041)], [(2.0, 5.0)], [(2.0, 5.0)], [(1.0, 5.0)], [(2.0, 5.0)], [(1.0, 1.5111553072929382)], [(1.0, 2.6998062133789062)], [(1.0, 3.7084461450576782)], [(2.0, 5.0)], [(1.661515235900879, 5.0)], [(1.0, 5.0)], [(2.0, 5.0)], [(2.0, 5.0)], [(1.0, 4.32735276222229)], [(1.0, 3.334682822227478)], [(1.0, 4.403512358665466)], [(2.0, 5.0)], [(2.0, 5.0)], [(2.0, 5.0)], [(2.0, 5.0)], [(1.934319019317627, 5.0)], [(1.0, 5.0)], [(2.0, 5.0)], [(1.0

fit: 100%|██████████| 100/100 [00:00<00:00, 1162.77it/s]


MSE=1.0566214115562695


Running experiments: 100%|██████████| 10/10 [15:36<00:00, 93.67s/it]

[[(2.0, 5.0)], [(1.0, 4.480564594268799)], [(1.0, 1.9503976106643677)], [(1.0, 4.597754240036011)], [(2.0, 5.0)], [(1.0, 5.0)], [(1.0, 4.959970951080322)], [(2.0, 5.0)], [(1.0, 5.0)], [(1.0, 5.0)], [(1.0, 2.3721492290496826)], [(2.0, 5.0)], [(1.0, 4.9160239696502686)], [(2.0, 5.0)], [(1.0, 2.1631381511688232)], [(1.0, 4.235215187072754)], [(2.235851287841797, 5.0)], [(2.0, 5.0)], [(1.0, 2.821466565132141)], [(2.0, 5.0)], [(1.0, 2.9113835096359253)], [(2.0, 5.0)], [(2.0, 5.0)], [(1.0, 2.073969841003418)], [(1.0, 5.0)], [(2.0, 5.0)], [(1.0, 4.98600172996521)], [(2.0, 5.0)], [(1.0, 4.411092042922974)], [(1.0, 5.0)], [(1.0, 2.28667950630188)], [(1.0, 2.5913678407669067)], [(1.0, 2.4975088834762573)], [(2.0, 5.0)], [(1.0972917079925537, 5.0)], [(2.0, 5.0)], [(2.0, 5.0)], [(2.0, 5.0)], [(1.8119382858276367, 5.0)], [(1.0, 1.9693706035614014)], [(2.0, 5.0)], [(1.0, 4.4431312084198)], [(2.0, 5.0)], [(1.0, 4.1024169921875)], [(1.0, 4.379819631576538)], [(2.0, 5.0)], [(2.0, 5.0)], [(1.0, 2.012997

In [6]:
mean_time, std_time, mean_memory/1024/1024, std_memory/1024/1024

(93.66908378999797, 2.824368985499139, 0.5525765419006348, 0.00562366076948688)